In [75]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer as stemmer
from tqdm.notebook import tqdm
import rule_book_functs as rbfuncts

In [4]:
incidents_fn = "data/source/20220413_D1_Incidents.csv"
rule_book_fn = "data/rule_book_kwic.csv"

In [5]:
rules = pd.read_csv(rule_book_fn)
incidents = pd.read_csv(incidents_fn, dtype=str)

In [6]:
incidents.rename(columns={'IncidentNumber': 'incident_id'}, inplace=True)
incidents['text'] = (
        incidents['ShortDescription'].astype(str).fillna('') + ' ' + 
        incidents['FullDescription'].astype(str).fillna('') + ' ' + 
        incidents['ImmediateAction'].astype(str).fillna('')
).str.lower()

incidents = incidents[['incident_id', 'text']]

In [5]:
#sample100 = incidents.sample(n=100)
#sample100.index = range(100)
#sample100.to_csv('sample100.csv', index=False)
sample100 = pd.read_csv('sample100.csv', dtype=str)

In [6]:
#incidents.text[0]

In [7]:
#test = incidents.text.head(1).apply(rbfuncts.lemmatize_text)
#print(list(test))

In [8]:
#test = incidents.text.head(1).apply(rbfuncts.stem_text)
#print(list(test))

In [9]:
#test_tokens = rbfuncts.tokenize(incidents.text[0])

In [10]:
#chk_text = 'employee was grinding with a four-inch grinder on an iron support in the compressor building when he felt discomfort to his left eye.'
#pattern = '(\\bemployee (.*) discomfort (.*) eye\\b)'
#rbfuncts.check_presence(pattern, chk_text)

In [11]:
#sents = sent_tokenize(incidents.text[0])
#for sent in sents:
#    if rbfuncts.check_presence(pattern, sent):
#        print(f'{rbfuncts.check_presence(pattern, sent)}: {sent}')

## Load Synonyms & Rules

In [12]:
syn_csv = pd.read_csv('synonyms.csv')

# Load the dictionary of synonyms
syn_dict = {}
for r in range(len(syn_csv)):
    syn_toks = syn_csv.keywords[r].split(',')
    syn_dict.update({syn_csv.syn[r]:syn_toks})
    
rul_csv = pd.read_csv('rules.csv')

In [13]:
finds_df = rbfuncts.rule_book_scan(sample100, syn_dict, rul_csv)

Checking rule 1 of 24 (eye injury or irritation)
14/100 [0] 0.0 hrs 0.0% classified (14.0% through)...

TypeError: slice indices must be integers or None or have an __index__ method

In [14]:
#finds_df

In [15]:
strs = 'This is me. He tripped and fell from the ladder.'
#strs = 'during the scaffolding elements transporting for the assembly of scaffolding scaffolder slipped and fell due to the slippery floor'
test = pd.DataFrame({'incident_id':[1052], 'text':[strs]})
rbfuncts.rule_book_scan(test, syn_dict, rul_csv)

Checking rule 1 of 24 (eye injury or irritation)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 2 of 24 (eye injury or irritation)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 3 of 24 (back injury)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 4 of 24 (covid)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 5 of 24 (fall from or working at height issue)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 6 of 24 (fall from or working at height issue)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 7 of 24 (fall from or working at height issue)
1/1 [0] 0.0 hrs 0.0% classified (100.0% through)...

Checking rule 8 of 24 (foot or leg injury)
1/1 [1] 0.0 hrs 100.0% classified (100.0% through)...

Checking rule 9 of 24 (general injury)
1/1 [1] 0.0 hrs 100.0% classified (100.0% through)...

Checking rule 10 of 24 (hand or arm injury)
1/1 [1] 0.0 hrs 100.0% classified (100.0% thro

TypeError: expected string or bytes-like object

In [ ]:
print(not False)

In [ ]:
# How many incidents were classified
len(list(dict.fromkeys(finds_df[finds_df.finds_list == True].incid_nums)))

In [ ]:
rbfuncts.deepdive_results(sample100, incidents, finds_df, focus='finds')

In [ ]:
rbfuncts.deepdive_results(sample100, incidents, finds_df, focus='misses')

In [ ]:
chk = 'the country gm and regional gm were on their way to their apartment when they were involved in a motor vehicle collision.'
#check_presence('hello(.*)vehicle(.*)collision', chk)
#check_presence('\bregional (.*) were (.*) way\b', chk)
#bool(re.search('\bregional (.*) were (.*) way\b', chk))
bool(re.search('involved(.*)motor(.*)collision', chk))

In [ ]:
org_string = "This is employee's hand"
pattern = r"'s"

# Replace all occurrences of character 's with an empty string
org_string = re.sub(pattern, '', org_string )
print(org_string)

In [27]:
def check_presence(pattern, string):
    if pattern:
        return bool(re.search(pattern, string))
    else:
        return False

In [ ]:
rul_csv

In [ ]:
x = 'car(.*)hit(.*)\\bcat'
pattern = x
check_presence(pattern, "the car hit the cat")

In [ ]:
re.sub(r"\\\\b", "\\b", 'no\\\\b')

In [ ]:
test = pd.DataFrame({'id':[1, 2, 2, 2, 3], 'cats':['apple', 'apple', 'apple', 'orange', 'kiwi']})
test

In [ ]:
test.groupby(['id'])['cats'].apply(', '.join).reset_index()

In [ ]:
'apple, apple, orange'.split(', ')

In [ ]:
list(dict.fromkeys('apple, apple, orange'.split(', ')))

In [ ]:
", ".join(list(dict.fromkeys('apple, apple, orange'.split(', '))))

In [ ]:
def remove_dups(ent):
    split_ent = ent.split(', ')
    uniqe_ent = list(dict.fromkeys(split_ent))
    rjoin_ent = ", ".join(uniqe_ent)
    return(rjoin_ent)

In [ ]:
remove_dups('apple, apple, orange')

In [ ]:
test = test.groupby(['id'])['cats'].apply(', '.join).reset_index()
test['cats'] = test['cats'].apply(lambda x: remove_dups(x))

In [ ]:
test

In [ ]:
for r in [1, 2, 3]:
    print(r)

In [ ]:
finds_count = 0
finds_count+=1
finds_count

In [ ]:
from time import sleep
import sys

for i in range(200):
    sys.stdout.write('\r')
    # the exact output you're looking for:
    sys.stdout.write(str(i))
    sys.stdout.flush()
    sleep(0.05)

In [ ]:
from datetime import datetime

now = datetime.now()


In [ ]:
then = datetime.now()

In [ ]:
diff = then - now

In [ ]:
round(diff.seconds/(60*60), 2)

In [ ]:
round(60*60/(60*60), 2)

In [ ]:
value = 10000
f'{value:,}'

In [ ]:
test = pd.DataFrame({'rule':['employee(.*)hurt(.*)hand', 'ip(.*)hurt(.*)hand']})

In [ ]:
test

In [ ]:
test['status'] = test['rule'].apply(lambda x: check_presence(x, 'the ip hurt his right hand'))
test

In [17]:
# Load raw incident data
incidents = pd.read_csv("data/source/20220413_D1_Incidents.csv", dtype=str)  

# Load the synonym dictionary in its raw csv format
syn_csv = pd.read_csv('synonyms.csv')

# Load the rule definitions
rul_csv = pd.read_csv('rules.csv')

# Concatenate some of the fields to make the 'text' field for searching
incidents.rename(columns={'IncidentNumber': 'incident_id'}, inplace=True)
incidents['text'] = (
        incidents['ShortDescription'].astype(str).fillna('') + ' ' + 
        incidents['FullDescription'].astype(str).fillna('') + ' ' + 
        incidents['ImmediateAction'].astype(str).fillna('')
).str.lower()

# We only need the incident ID and the text for now
incidents = incidents[['incident_id', 'text']]

# Now convert the csv format into a dictionary of synonyms
syn_dict = {}
for r in range(len(syn_csv)):
    syn_toks = syn_csv.keywords[r].split(',')
    syn_dict.update({syn_csv.syn[r]:syn_toks})

In [18]:
rul_csv

,rule,keyword,syns,voids,span,shuffle
0,eye injury or irritation,eye,"worker_syns, pain_syns, eye_syns",NaN,8.0,True
1,eye injury or irritation,eye,"gen1_syns, pain_syns, eye_syns",NaN,8.0,True
2,back injury,back,"worker_syns, back_pain_syns, back_syns",NaN,8.0,True
3,covid,-,"worker_syns, tested_syns, covid_syns",NaN,8.0,True
4,fall from or working at height issue,height,"height_equipment_syns, falling_object_syns","same level, slippery floor, wet floor, slipper...",8.0,False
5,fall from or working at height issue,fall,"height_equipment_syns, falling_object_syns","same level, slippery floor, wet floor, slipper...",8.0,False
6,fall from or working at height issue,-,"worker_syns, height_equipment_syns, falling_ob...","same level, slippery floor, wet floor, slipper...",8.0,True
7,foot or leg injury,-,"worker_syns, hurt_syns, leg_syns",NaN,8.0,True
8,general injury,-,"worker_syns, hurt_syns, gen_body_syns",NaN,8.0,True
9,hand or arm injury,-,"worker_syns, hurt_syns, arm_syns",NaN,8.0,True


In [19]:
# worker_syns, pain_syns, eye_syns
worker_syns = ['employee', 'ip', 'worker', 'man', 'woman', 'he', 'she', 'his']
pain_syns = ['pain', 'discomfort', 'soreness', 'hurt', 'something in', 'grit', 'swelling']
eye_syns = ['eye', 'pupil', 'iris']

In [20]:
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [tok for tok in tokens if tok!='-']
    return tokens

In [21]:
rul_syns = tokenize(re.sub(r", ", " ", rul_csv.syns[0]))
rul_syns

['worker_syns', 'pain_syns', 'eye_syns']

In [22]:
syn_dict.get(rul_syns[0])

['employee',
 ' ip',
 ' worker',
 ' man',
 ' woman',
 ' he',
 ' she',
 ' his',
 ' her',
 ' left',
 ' right',
 ' operator',
 ' operative',
 ' technician',
 ' scaffolder',
 ' team member',
 ' person']

In [125]:
import itertools

c = 1

patterns = []
lhs = []
rhs = []
syn_count = len(rul_syns)

for i in range(syn_count):
    locals()[f'l{i+1}'] = syn_dict.get(rul_syns[i])

iter_prod = itertools.product(l1, l2, l3)

for r in iter_prod: 
    if syn_count == 2:
        pt = f'{r[0].strip()}(.*){r[1].strip()}'
    elif syn_count == 3:
        pt = f'{r[0].strip()}(.*){r[1].strip()}(.*){r[2].strip()}'
    else:
        pt = f'{r[0].strip()}(.*){r[1].strip()}(.*){r[2].strip()}(.*){r[3].strip()}'
    patterns.append(pt)
    lhs.append(r[0].strip())
    rhs.append(r[2].strip())
    c += 1
    
test = pd.DataFrame({'pattern':patterns})    
test['lhs'] = lhs
test['rhs'] = rhs

In [126]:
test

,pattern,lhs,rhs
0,employee(.*)pain(.*)eye,employee,eye
1,employee(.*)pain(.*)pupil,employee,pupil
2,employee(.*)pain(.*)iris,employee,iris
3,employee(.*)discomfort(.*)eye,employee,eye
4,employee(.*)discomfort(.*)pupil,employee,pupil
...,...,...,...
1321,person(.*)swelling(.*)pupil,person,pupil
1322,person(.*)swelling(.*)iris,person,iris
1323,person(.*)swollen(.*)eye,person,eye
1324,person(.*)swollen(.*)pupil,person,pupil


In [127]:
chk_sent = 'the person hurt his right eye'
sum(test['lhs'].apply(lambda x: find_index(chk_sent, x, pos='left')))

936

In [129]:
chk_sent = 'the person hurt his right eye'
split_chk_sent = chk_sent.split()
print(split_chk_sent)
test['found'] = test['pattern'].apply(lambda x: check_presence(x, chk_sent))
test['lhs_index'] = test['lhs'].apply(lambda x: find_index(chk_sent, x, pos='left'))
test['rhs_index'] = test['rhs'].apply(lambda x: find_index(chk_sent, x, pos='right'))
test['span'] = test['rhs_index'] - test['lhs_index']
test

['the', 'person', 'hurt', 'his', 'right', 'eye']


,pattern,lhs,rhs,found,lhs_index,rhs_index,span
0,employee(.*)pain(.*)eye,employee,eye,False,0,6,6
1,employee(.*)pain(.*)pupil,employee,pupil,False,0,0,0
2,employee(.*)pain(.*)iris,employee,iris,False,0,0,0
3,employee(.*)discomfort(.*)eye,employee,eye,False,0,6,6
4,employee(.*)discomfort(.*)pupil,employee,pupil,False,0,0,0
...,...,...,...,...,...,...,...
1321,person(.*)swelling(.*)pupil,person,pupil,False,2,0,-2
1322,person(.*)swelling(.*)iris,person,iris,False,2,0,-2
1323,person(.*)swollen(.*)eye,person,eye,False,2,6,4
1324,person(.*)swollen(.*)pupil,person,pupil,False,2,0,-2


In [132]:
test[test['found'] == True]

,pattern,lhs,rhs,found,lhs_index,rhs_index,span
399,he(.*)hurt(.*)eye,he,eye,True,1,6,5
1257,person(.*)hurt(.*)eye,person,eye,True,2,6,4


In [133]:
test[test['lhs_index'] != 0]

,pattern,lhs,rhs,found,lhs_index,rhs_index,span
390,he(.*)pain(.*)eye,he,eye,False,1,6,5
391,he(.*)pain(.*)pupil,he,pupil,False,1,0,-1
392,he(.*)pain(.*)iris,he,iris,False,1,0,-1
393,he(.*)discomfort(.*)eye,he,eye,False,1,6,5
394,he(.*)discomfort(.*)pupil,he,pupil,False,1,0,-1
...,...,...,...,...,...,...,...
1321,person(.*)swelling(.*)pupil,person,pupil,False,2,0,-2
1322,person(.*)swelling(.*)iris,person,iris,False,2,0,-2
1323,person(.*)swollen(.*)eye,person,eye,False,2,6,4
1324,person(.*)swollen(.*)pupil,person,pupil,False,2,0,-2


In [88]:
chk_sent = 'the person hurt his right hurts eye'
split_chk_sent = chk_sent.split()
print(split_chk_sent)

['the', 'person', 'hurt', 'his', 'right', 'hurts', 'eye']


In [89]:
def find_index(chk_string, keyword, pos='left'):
    try:
        split_chk = chk_string.split()
        #find_index = max(loc for loc, val in enumerate(split_chk) if val == keyword) + 1
        find_index = [loc for loc, val in enumerate(split_chk) if bool(re.search(keyword, val))]
        if pos == 'left':
            find_index = min(find_index)+1
        else:
            find_index = max(find_index)+1
    except:
        print
        find_index = 0
    return(find_index)

find_index(chk_sent, 'person', pos='left')

2

In [14]:
keyword = 'leg'
window = 2
pre_context = "(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,<window>}<keyword>".replace('<keyword>', str(keyword)).replace('<window>', str(window))
post_context = "<keyword>(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,<window>}".replace('<keyword>', str(keyword)).replace('<window>', str(window))
all_context = "(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,<window>}<keyword>(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,<window>}".replace('<keyword>', str(keyword)).replace('<window>', str(window))
pre_context

"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,2}leg"

In [15]:
doc = 'My big fat leg is really broke'
re.findall(pre_context, doc)

['big fat leg']

In [16]:
re.findall(post_context, doc)

['leg is really']

In [23]:
"<keyword> how you doing".replace('<keyword>', str(keyword))

'leg how you doing'

In [53]:
rules = pd.read_csv('data/rule_book.csv')
rules["keyword"] = [x.replace("*", "[a-zA-Z'-]*") + r"\b" for x in rules["keyword"]]
rules["rules_pre"] = [rbfuncts.translate_to_regex(x) for x in rules["rules_pre"]]
rules["rules_post"] = [rbfuncts.translate_to_regex(x) for x in rules["rules_post"]]
rules["rules_all"] = [rbfuncts.translate_to_regex(x) for x in rules["rules_all"]]
rules["voids"] = [rbfuncts.translate_to_regex(x) for x in rules["voids"]]
categories = list(set(rules["group"]))
cat_dict = {cat: i for i, cat in enumerate(categories)}
category_indicators = [False] * len(categories)

In [54]:
categories

['minor non-injury event',
 'fuel spill or leak',
 'non-work related injury or illness',
 'competency',
 'slips & trips',
 'foot or leg injury',
 'sharp objects',
 'hand or arm injury',
 'disease or infection',
 'alarms',
 'minor fluid spill',
 'knife incident',
 'aggressive behaviour',
 'confined space non-conformance or issue',
 'back injury',
 'dropped object or material',
 'eye injury or irritation',
 'line strike',
 'covid',
 'bites (animals or insects)',
 'fall from or working at height issue',
 'general injury',
 'lifting or moving loads',
 'mechanical or equipment issue',
 'site compliance or practice issue',
 'weather related issue',
 'security or general safety concern',
 'vehicle Incident involving animal',
 'fire',
 'burns',
 'work vehicle incident',
 'ppe non-compliance',
 'hydraulic fluid or oil leak',
 'alcohol or drugs',
 'theft',
 'vandalism',
 'environmental leak or issue',
 'hazardous materials or work',
 'line of fire',
 'excavations',
 'speeding',
 'general illness

In [55]:
cat_dict

{'minor non-injury event': 0,
 'fuel spill or leak': 1,
 'non-work related injury or illness': 2,
 'competency': 3,
 'slips & trips': 4,
 'foot or leg injury': 5,
 'sharp objects': 6,
 'hand or arm injury': 7,
 'disease or infection': 8,
 'alarms': 9,
 'minor fluid spill': 10,
 'knife incident': 11,
 'aggressive behaviour': 12,
 'confined space non-conformance or issue': 13,
 'back injury': 14,
 'dropped object or material': 15,
 'eye injury or irritation': 16,
 'line strike': 17,
 'covid': 18,
 'bites (animals or insects)': 19,
 'fall from or working at height issue': 20,
 'general injury': 21,
 'lifting or moving loads': 22,
 'mechanical or equipment issue': 23,
 'site compliance or practice issue': 24,
 'weather related issue': 25,
 'security or general safety concern': 26,
 'vehicle Incident involving animal': 27,
 'fire': 28,
 'burns': 29,
 'work vehicle incident': 30,
 'ppe non-compliance': 31,
 'hydraulic fluid or oil leak': 32,
 'alcohol or drugs': 33,
 'theft': 34,
 'vandalism

In [7]:
def check_presence(pattern, string):
    if pattern:
        return bool(re.search(pattern, string))
    else:
        return False

In [8]:
def find_pattern(doc, keyword, check_pre, check_post, check_all, check_void, window):
    """
    for a list of tokens finds specified keyword and returns True
    if the neighbourhood of this keyword satisfies pre-, post- or all- context rules
    and doesn't contain anything forbidden
    :param tokens: list of tokens
    :param keyword: pattern which a token should match
    :param check_pre: pattern which several previous tokens (concatenated) should match
    :param check_post: pattern which several subsequent tokens (concatenated) should match
    :param check_all: pattern which previous tokens + keyword + subsequent tokens should match
    :param check_void: pattern which previous tokens + keyword + subsequent tokens should NOT match
    :param window: N of pre and post tokens to consider
    :return: True/False - whether at least one matching part was found
    """
    # extract contexts of keyword (if any found)
    # check if keyword in sentence
    any_match = re.search(keyword, doc)

    if any_match is None:
        return False
    else:
        # We want to finad and extract the context of the keyword in the sentence
        # With a predefined window
        pre_context = "(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,<window>}<keyword>".replace('<keyword>', str(keyword)).replace('<window>', str(window))
        post_context = "<keyword>(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,<window>}".replace('<keyword>', str(keyword)).replace('<window>', str(window))
        all_context = "(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,<window>}<keyword>(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,<window>}".replace('<keyword>', str(keyword)).replace('<window>', str(window))
        
        pre_match = re.findall(pre_context, doc)
        post_match = re.findall(post_context, doc)
        all_match = re.findall(all_context, doc)

    # Check if any of the given sentences is on list of pre/post/all keywords.
    pre_check = any([check_presence(check_pre, pre) for pre in pre_match])
    post_check = any([check_presence(check_post, post) for post in post_match])
    all_check = any([check_presence(check_all, all_) for all_ in all_match])
    void_check = any([check_presence(check_void, all_) for all_ in all_match])

    # Perform final tests
    final_match = (pre_check or post_check or all_check) and not void_check
    return final_match

In [58]:
num_rules = len(rules['group'])
window = 12
incidents = pd.read_csv("data/source/20220413_D1_Incidents.csv", dtype=str)  
# Concatenate some of the fields to make the 'text' field for searching
incidents.rename(columns={'IncidentNumber': 'incident_id'}, inplace=True)
incidents['text'] = (
        incidents['ShortDescription'].astype(str).fillna('') + ' ' + 
        incidents['FullDescription'].astype(str).fillna('') + ' ' + 
        incidents['ImmediateAction'].astype(str).fillna('')
).str.lower()

# We only need the incident ID and the text for now
incidents = incidents[['incident_id', 'text']]
sample100 = incidents.sample(100)
doc=sample100

In [59]:
doc

,incident_id,text
5689,20164451,"pen slipped from a height of 4m, however it la..."
5854,20164286,vehicle es 130 entered in contact with the fen...
16030,20160147,person entered barricaded area with no authori...
17373,20201682,ip suffered pulled muscle in left shoulder dur...
5427,20164715,supervisor work instruction not followed migra...
...,...,...
8184,20192283,contained visqueen fire two turner employees w...
27065,20222136,a wood employee tested positive for covid-19 ...
2052,20182542,an employee had fuel contact his right eye whi...
18601,20202943,a wood heavy civils employee was involved in a...


In [60]:
rules

,group,keyword,rules_pre,rules_post,rules_all,voids
0,slips & trips,slip[a-zA-Z'-]*\b,(\bwalking (.*) snow\b)|(\bwalking (.*) ice\b)...,(\bfall\b)|(\bfell\b)|(\bwent over\b)|(\bhurt....,(\bfall\b)|(\bfell\b)|(\btrip\b)|(\btripped\b)...,(\bfell (.*) bike\b)|(\bno slip\b)|(\bno fall\...
1,slips & trips,trip[a-zA-Z'-]*\b,(\bwalking (.*) snow\b)|(\bwalking (.*) ice\b)...,(\bfall\b)|(\bfell\b)|(\bwent over\b)|(\bfraye...,(\bfall\b)|(\bfell\b)|(\btrip\b)|(\btripped\b)...,(\bfell (.*) bike\b)|(\bno slip\b)|(\bno fall\...
2,slips & trips,stumble[a-zA-Z'-]*\b,(\bcausing.*cleaner to\b)|(\bcausing.*ip to\b)...,(\bfloor\b)|(\band twist.*\b),(\bhurting foot\b),
3,slips & trips,uneven\b,(\bcausing.*cleaner to\b)|(\bcausing.*ip to\b)...,(\band twist.*\b),(\bemployee.* uneven.* surface\b)|(\bip.* unev...,
4,slips & trips,fell\b,(\bcausing.*cleaner to\b)|(\bcausing.*ip to\b)...,(\band twist.*\b)|(\bon.*floor\b)|(\bon.*groun...,(\bemployee fell back.*\b)|(\bip fell back.*\b...,(\bstraps\b)|(\bdrop.*\b)|(\bfell (.*) bike\b)...
...,...,...,...,...,...,...
315,office or facilities incident or injury,toilet\b,(\bflood.*\b),(\boverflow\b)|(\bflood.*\b)|(\bleak.*\b),,
316,vandalism,sever[a-zA-Z'-]*\b,,,(\bintentionally.*sever.*others\b),
317,line of fire,pressure\b,,,(\btight.*under.*pressure_tight.*while.*pressu...,
318,line of fire,line\b,,,(\bline of fire\b),


In [9]:
finds = [
        (
            rules['group'][j],
            find_pattern(
                doc='employee’s hand was caught in between a flang. He then got in the car and drove into a deer',
                keyword=rules["keyword"][j],
                check_pre=rules["rules_pre"][j],
                check_post=rules["rules_post"][j],
                check_all=rules["rules_all"][j],
                check_void=rules["voids"][j],
                window=window,
            )
        ) for j in range(num_rules)
    ]

NameError: name 'num_rules' is not defined

In [10]:
finds

NameError: name 'finds' is not defined

In [69]:
output = list(filter(lambda x: x[1], finds))
output

[('vehicle Incident involving animal', True),
 ('vehicle incident general', True),
 ('hand or arm injury', True)]

In [70]:
output = list(set([x[0] for x in output]))
output

['hand or arm injury',
 'vehicle incident general',
 'vehicle Incident involving animal']

In [37]:
rules = pd.read_csv('data/rule_book_kwic.csv')

In [38]:
def translate_to_regex(rule_part):
    """
    converts rule_book rules to regex format;
    drops trailing separators in process
    """
    if isinstance(rule_part, str):
        rule_part = re.sub(r"\s{2,}", " ", rule_part)
        rule_part = re.sub(r'\\\\b', r'\\b', rule_part)
        
        return r'(\b' + r'\b)|(\b'.join([s for s in rule_part.split('_ ') if s]) + r'\b)'
    else:
        return ''

In [39]:
# Transform columns to regular expression. 
rules["keyword"] = [x.replace("*", "[a-zA-Z'-]*") + r"\b" for x in rules["keyword"]]
rules["rules_pre"] = [translate_to_regex(x) for x in rules["rules_pre"]]
rules["rules_post"] = [translate_to_regex(x) for x in rules["rules_post"]]
rules["rules_all"] = [translate_to_regex(x) for x in rules["rules_all"]]
rules["voids"] = [translate_to_regex(x) for x in rules["voids"]]

In [50]:
rules.iloc[28, 1:2]

keyword    -\b
Name: 28, dtype: object

In [53]:
rules['group'][10]

'vehicle Incident involving animal'

In [31]:
set(list(rules.group))

{'aggressive behaviour',
 'alarms',
 'alcohol or drugs',
 'back injury',
 'bites (animals or insects)',
 'burns',
 'competency',
 'confined space non-conformance or issue',
 'covid',
 'disease or infection',
 'dropped object or material',
 'electric shock or issue',
 'environmental leak or issue',
 'excavations',
 'eye injury or irritation',
 'fall from or working at height issue',
 'fire',
 'foot or leg injury',
 'fuel spill or leak',
 'general illness or health issue',
 'general injury',
 'hand or arm injury',
 'hazardous materials or work',
 'head or face injury',
 'hydraulic fluid or oil leak',
 'knife incident',
 'lifting or moving loads',
 'line of fire',
 'line strike',
 'mechanical or equipment issue',
 'minor fluid spill',
 'minor non-injury event',
 'non-work related injury or illness',
 'office or facilities incident or injury',
 'offshore transfer',
 'ppe non-compliance',
 'security or general safety concern',
 'sharp objects',
 'site compliance or practice issue',
 'slips 

In [25]:
finds = [
        (
            rules['group'][j],
            find_pattern(
                doc='pedestrian struck by forward-moving vehicle in roadway an employee stepped off the back of a truck and was struck by an oncoming vehicle. the employee suffered fractured ribs and bruised lungs and a fractured leg.',
                keyword=rules["keyword"][j],
                check_pre=rules["rules_pre"][j],
                check_post=rules["rules_post"][j],
                check_all=rules["rules_all"][j],
                check_void=rules["voids"][j],
                window=12,
            )
        ) for j in range(len(rules))
    ]

In [32]:
tmp = set(list(rules.group))
for t in tmp:
    print(t)

ppe non-compliance
hydraulic fluid or oil leak
disease or infection
vehicle incident general
alcohol or drugs
line strike
hazardous materials or work
covid
security or general safety concern
minor non-injury event
foot or leg injury
competency
fall from or working at height issue
burns
minor fluid spill
weather related issue
general injury
line of fire
alarms
vandalism
hand or arm injury
confined space non-conformance or issue
lifting or moving loads
bites (animals or insects)
mechanical or equipment issue
speeding
electric shock or issue
eye injury or irritation
non-work related injury or illness
fire
offshore transfer
sharp objects
environmental leak or issue
knife incident
head or face injury
office or facilities incident or injury
vehicle Incident involving animal
fuel spill or leak
slips & trips
excavations
back injury
general illness or health issue
site compliance or practice issue
theft
dropped object or material
aggressive behaviour
work vehicle incident


In [26]:
finds

[('slips & trips', False),
 ('slips & trips', False),
 ('slips & trips', False),
 ('slips & trips', False),
 ('slips & trips', False),
 ('slips & trips', False),
 ('eye injury or irritation', False),
 ('eye injury or irritation', False),
 ('eye injury or irritation', False),
 ('vehicle Incident involving animal', False),
 ('vehicle Incident involving animal', False),
 ('vehicle Incident involving animal', False),
 ('vehicle Incident involving animal', False),
 ('theft', False),
 ('theft', False),
 ('theft', False),
 ('theft', False),
 ('theft', False),
 ('theft', False),
 ('theft', False),
 ('theft', False),
 ('competency', False),
 ('covid', False),
 ('covid', False),
 ('covid', False),
 ('covid', False),
 ('covid', False),
 ('covid', False),
 ('back injury', False),
 ('back injury', False),
 ('back injury', False),
 ('fall from or working at height issue', False),
 ('fall from or working at height issue', False),
 ('fall from or working at height issue', False),
 ('fall from or worki

In [27]:
list(filter(lambda x: x[1], finds))

[('foot or leg injury', True),
 ('general injury', True),
 ('lifting or moving loads', True)]

In [97]:
rules.rules_all[0]

'(\\bfp.*eye.*{ip}\\b)'

In [98]:
check_presence(rules.rules_all[0], 'hello fp in the eye of the ip my')

False

In [90]:
check_presence('(\\bfp.*eye.*ip\\b)', 'fp in the eye of the ip')

True

In [79]:
bool(re.search('(\bfp.*eye.*ip\b)', 'fp in the eye of the ip'))

False

In [100]:
translate_to_regex(rules.rules_all[0])

'(\\bfp.*eye.*{ip}\\b)'

In [106]:
rules.rules_all[0]

'fp.*eye.*{ip}'

In [159]:
bool(re.search("{[a-zA-Z'-]*}", rules.rules_all[0]))

True

In [160]:
bool(re.search("{[a-zA-Z'-]*}", 'fp.*eye.*ip'))

False

In [198]:
re.sub("{[a-zA-Z'-]*}", 'hello', 'fp.*eye.*{worker}')

'fp.*eye.*hello'

In [286]:
syns_db = pd.read_csv('synonyms.csv')
syns_db

,syn,keywords
0,arm_syns,"arm, elbow, forearm, wrist, palm, finger, thum..."
1,back_pain_syns,"pain, discomfort, soreness, hurt, swelling, s..."
2,back_syns,back
3,collision_syns,"collision, collission, accident, clipped"
4,competency_syns,"training, competency, certification, certifica..."
5,covid_syns,"covid, covid-19, coronavirus, corona virus, pcr"
6,eye_syns,"eye, pupil, iris"
7,failure_syns,"failed, failure, fail, leaked, leak, rupture, ..."
8,fall_syns,"fell, twisted, fall"
9,falling_object_syns,"fell, fall, falling, dropped"


In [343]:
def replace_syns(in_str):
    mod_str = re.sub('_syns', '', in_str) 
    return(mod_str)

replace_syns('worker_syns')
syns_db['syn'] = syns_db['syn'].apply(replace_syns)
syns_db

,syn,keywords
0,arm,"arm, elbow, forearm, wrist, palm, finger, thum..."
1,back_pain,"pain, discomfort, soreness, hurt, swelling, s..."
2,back,back
3,collision,"collision, collission, accident, clipped"
4,competency,"training, competency, certification, certifica..."
5,covid,"covid, covid-19, coronavirus, corona virus, pcr"
6,eye,"eye, pupil, iris"
7,failure,"failed, failure, fail, leaked, leak, rupture, ..."
8,fall,"fell, twisted, fall"
9,falling_object,"fell, fall, falling, dropped"


In [384]:
rules = pd.read_csv('data/rule_book_test.csv')
scan_rules = rules.rules_all[0].split('_ ')
scan_rules

['fp.*eye.*{worker}', 'spanner in {eye}', '{pain}.*{eye}.*{worker}']

In [350]:
re.findall("{[a-zA-Z'-]*}", '{spanner} in {eye}')

['{spanner}', '{eye}']

In [351]:
def mutate_syn(syns, syns_db, rule, num):
    syn_rules = []
    if num == 1:    
        syn_raw = syns[0]
        syn = re.sub(r"[\([{})\]]", '', syn_raw)
        found_syn = syns_db.loc[(syns_db['syn'] == syn)]
        for keyword in found_syn.iloc[0, 1].split(', '):
            syn_rules.append(re.sub(syn_raw, keyword, rule))
        syn_rules = '_ '.join(syn_rules)
        return(syn_rules)
    elif num == 2:
        syn_raw_1 = syns[0]
        syn_1 = re.sub(r"[\([{})\]]", '', syn_raw_1)
        found_syn_1 = syns_db.loc[(syns_db['syn'] == syn_1)]

        syn_raw_2 = syns[1]
        syn_2 = re.sub(r"[\([{})\]]", '', syn_raw_2)
        found_syn_2 = syns_db.loc[(syns_db['syn'] == syn_2)]

        for keyword_1 in found_syn_1.iloc[0, 1].split(', '):
            for keyword_2 in found_syn_2.iloc[0, 1].split(', '):
                pass_1 = re.sub(syn_raw_1, keyword_1, rule)
                pass_2 = re.sub(syn_raw_2, keyword_2, pass_1)
                syn_rules.append(pass_2)
        syn_rules = '_ '.join(syn_rules)
        return(syn_rules)
    elif num == 3:
        syn_raw_1 = syns[0]
        syn_1 = re.sub(r"[\([{})\]]", '', syn_raw_1)
        found_syn_1 = syns_db.loc[(syns_db['syn'] == syn_1)]

        syn_raw_2 = syns[1]
        syn_2 = re.sub(r"[\([{})\]]", '', syn_raw_2)
        found_syn_2 = syns_db.loc[(syns_db['syn'] == syn_2)]

        syn_raw_3 = syns[2]
        syn_3 = re.sub(r"[\([{})\]]", '', syn_raw_3)
        found_syn_3 = syns_db.loc[(syns_db['syn'] == syn_3)]

        for keyword_1 in found_syn_1.iloc[0, 1].split(', '):
            for keyword_2 in found_syn_2.iloc[0, 1].split(', '):
                for keyword_3 in found_syn_3.iloc[0, 1].split(', '):
                    pass_1 = re.sub(syn_raw_1, keyword_1, rule)
                    pass_2 = re.sub(syn_raw_2, keyword_2, pass_1)
                    pass_3 = re.sub(syn_raw_3, keyword_3, pass_2)
                    syn_rules.append(pass_3)
        syn_rules = '_ '.join(syn_rules)
        return(syn_rules)        

In [352]:
scan_rules = rules.rules_all[0].split('_ ')
out_rules = []

for rule in scan_rules:
    syns = re.findall("{[a-zA-Z'-]*}", rule)
    syn_count = len(syns)
    if syn_count == 1:
        syn_rules = mutate_syn(syns, syns_db, rule, 1)
        out_rules.append(syn_rules)
    elif syn_count == 2:
        syn_rules = mutate_syn(syns, syns_db, rule, 2)
        out_rules.append(syn_rules)
    elif syn_count == 3:
        syn_rules = mutate_syn(syns, syns_db, rule, 3)
        out_rules.append(syn_rules)
    else:       
        out_rules.append(rule)
out_rules = '_ '.join(out_rules)
print(out_rules)

fp.*eye.*employee_ fp.*eye.*ip_ fp.*eye.*worker_ fp.*eye.*man_ fp.*eye.*woman_ fp.*eye.*he_ fp.*eye.*she_ fp.*eye.*his_ fp.*eye.*her_ fp.*eye.*left_ fp.*eye.*right_ fp.*eye.*operator_ fp.*eye.*operative_ fp.*eye.*technician_ fp.*eye.*scaffolder_ fp.*eye.*team member_ fp.*eye.*person_ spanner in eye_ spanner in pupil_ spanner in iris_ pain.*eye.*employee_ pain.*eye.*ip_ pain.*eye.*worker_ pain.*eye.*man_ pain.*eye.*woman_ pain.*eye.*he_ pain.*eye.*she_ pain.*eye.*his_ pain.*eye.*her_ pain.*eye.*left_ pain.*eye.*right_ pain.*eye.*operator_ pain.*eye.*operative_ pain.*eye.*technician_ pain.*eye.*scaffolder_ pain.*eye.*team member_ pain.*eye.*person_ pain.*pupil.*employee_ pain.*pupil.*ip_ pain.*pupil.*worker_ pain.*pupil.*man_ pain.*pupil.*woman_ pain.*pupil.*he_ pain.*pupil.*she_ pain.*pupil.*his_ pain.*pupil.*her_ pain.*pupil.*left_ pain.*pupil.*right_ pain.*pupil.*operator_ pain.*pupil.*operative_ pain.*pupil.*technician_ pain.*pupil.*scaffolder_ pain.*pupil.*team member_ pain.*pupil.*

In [353]:
def translate_to_regex(rule_part, syns_db):
    """
    converts rule_book rules to regex format;
    drops trailing separators in process
    """
    if isinstance(rule_part, str): # If rule part is a string...
        
        split_rule = rule_part.split('_ ')
        out_rule = []

        for rule in split_rule:
            syns = re.findall("{[a-zA-Z'-]*}", rule)
            syn_count = len(syns)
            if syn_count == 1:
                syn_rules = mutate_syn(syns, syns_db, rule, 1)
                out_rule.append(syn_rules)
            elif syn_count == 2:
                syn_rules = mutate_syn(syns, syns_db, rule, 2)
                out_rule.append(syn_rules)
            elif syn_count == 3:
                syn_rules = mutate_syn(syns, syns_db, rule, 3)
                out_rule.append(syn_rules)
            else:       
                out_rule.append(rule)
        out_rule = '_ '.join(out_rule)
        
        rule_part = re.sub(r"\s{2,}", " ", out_rule)
        rule_part = re.sub(r'\\\\b', r'\\b', rule_part)
        
        return r'(\b' + r'\b)|(\b'.join([s for s in rule_part.split('_ ') if s]) + r'\b)'
    else:
        return '' 

In [356]:
chk = translate_to_regex(rules.rules_all[0], syns_db)
chk

'(\\bfp.*eye.*employee\\b)|(\\bfp.*eye.*ip\\b)|(\\bfp.*eye.*worker\\b)|(\\bfp.*eye.*man\\b)|(\\bfp.*eye.*woman\\b)|(\\bfp.*eye.*he\\b)|(\\bfp.*eye.*she\\b)|(\\bfp.*eye.*his\\b)|(\\bfp.*eye.*her\\b)|(\\bfp.*eye.*left\\b)|(\\bfp.*eye.*right\\b)|(\\bfp.*eye.*operator\\b)|(\\bfp.*eye.*operative\\b)|(\\bfp.*eye.*technician\\b)|(\\bfp.*eye.*scaffolder\\b)|(\\bfp.*eye.*team member\\b)|(\\bfp.*eye.*person\\b)|(\\bspanner in eye\\b)|(\\bspanner in pupil\\b)|(\\bspanner in iris\\b)|(\\bpain.*eye.*employee\\b)|(\\bpain.*eye.*ip\\b)|(\\bpain.*eye.*worker\\b)|(\\bpain.*eye.*man\\b)|(\\bpain.*eye.*woman\\b)|(\\bpain.*eye.*he\\b)|(\\bpain.*eye.*she\\b)|(\\bpain.*eye.*his\\b)|(\\bpain.*eye.*her\\b)|(\\bpain.*eye.*left\\b)|(\\bpain.*eye.*right\\b)|(\\bpain.*eye.*operator\\b)|(\\bpain.*eye.*operative\\b)|(\\bpain.*eye.*technician\\b)|(\\bpain.*eye.*scaffolder\\b)|(\\bpain.*eye.*team member\\b)|(\\bpain.*eye.*person\\b)|(\\bpain.*pupil.*employee\\b)|(\\bpain.*pupil.*ip\\b)|(\\bpain.*pupil.*worker\\b)|(\\

In [359]:
check_presence(chk, 'An fp was lodged in the left eye of the new technician.')

True

In [360]:
import itertools
load_cases = []
for r in itertools.product(['hell', 'butts'], ['me', 'you','them']): 
    lc = f'{r[0]}.*{r[1]}'
    load_cases.append(lc)
load_cases

['hell.*me',
 'hell.*you',
 'hell.*them',
 'butts.*me',
 'butts.*you',
 'butts.*them']

In [361]:
test_rule = translate_to_regex(syn_rules)

TypeError: translate_to_regex() missing 1 required positional argument: 'syns_db'

In [215]:
check_presence(test_rule, 'hello fp in the eye of the operator of the ip my')

True

In [375]:
osha_incs = pd.read_csv("data/source/OSHA_January2015toJuly2021.csv", dtype=str)

In [376]:
osha_incs.columns

Index(['ID', 'UPA', 'EventDate', 'Employer', 'Address1', 'Address2', 'City',
       'State', 'Zip', 'Latitude', 'Longitude', 'Primary NAICS',
       'Hospitalized', 'Amputation', 'Inspection', 'Final Narrative', 'Nature',
       'NatureTitle', 'Part of Body', 'Part of Body Title', 'Event',
       'EventTitle', 'Source', 'SourceTitle', 'Secondary Source',
       'Secondary Source Title'],
      dtype='object')

In [377]:
osha_incs['Final Narrative']

0        Three correctional facility guards were escort...
1        Employee in the Machine Shop received second d...
2        A truck driver fell approximately 4 feet while...
3        An employee's leg was pinned between a truck a...
4        An employee working on the Line 6 Auto-Beller ...
                               ...                        
66694    An employee was using an extension ladder whil...
66695    An employee was swinging a hammer while on a 6...
66696    On July 31, 2021, at 11:30 a.m., an employee w...
66697    An employee accessed an electrical panel to sw...
66698    A nurse was sitting at a nurse's station. A pa...
Name: Final Narrative, Length: 66699, dtype: object

In [378]:
# Concatenate some of the fields to make the 'text' field for searching
osha_incs.rename(columns={'ID': 'incident_id'}, inplace=True)
osha_incs['text'] = (
        osha_incs['EventTitle'].astype(str).fillna('') + ' ' + 
        osha_incs['Final Narrative'].astype(str).fillna('')
).str.lower()
osha_incs['dataset'] = 'OSHA'

In [379]:
osha_incs = osha_incs[['incident_id', 'dataset', 'text']]
osha_incs

,incident_id,dataset,text
0,2015010015,OSHA,injured by physical contact with person while ...
1,2015010016,OSHA,"ignition of vapors, gases, or liquids employee..."
2,2015010018,OSHA,other fall to lower level less than 6 feet a t...
3,2015010019,OSHA,caught in or compressed by equipment or object...
4,2015010020,OSHA,caught in running equipment or machinery durin...
...,...,...,...
66694,2021076378,OSHA,fall on water vehicle an employee was using an...
66695,2021076379,OSHA,"other fall to lower level, unspecified an empl..."
66696,2021076380,OSHA,injured by slipping or swinging object held by...
66697,2021076384,OSHA,"direct exposure to electricity, greater than 2..."


In [310]:
rules = pd.read_csv('data/rule_book_kwic.csv')
focus_group = 'competency'
rules

,group,keyword,rules_pre,rules_post,rules_all,voids
0,slips & trips,slip*,walking (.*) snow_ walking (.*) ice_ walking (...,fall_ fell_ went over_ hurt.*_ and twist.*_ to...,{slip}.*{fall}_ fall_ fell_ trip_ tripped_ ind...,fell (.*) bike_ no slip_ no fall_ it slipped_ ...
1,slips & trips,trip*,walking (.*) snow_ walking (.*) ice_ walking (...,fall_ fell_ went over_ frayed_ loose_ hazard__...,fall_ fell_ trip_ tripped_ individual slipped_...,fell (.*) bike_ no slip_ no fall_ it slipped_ ...
2,slips & trips,stumble*,causing.*cleaner to_ causing.*ip to_ causing.*...,floor_ and twist.*,hurting foot,rung.*step.*_ while sitting
3,slips & trips,uneven,causing.*cleaner to_ causing.*ip to_ causing.*...,_ and twist.*,employee.* uneven.* surface_ ip.* uneven.* sur...,rung.*step.*_ while sitting
4,slips & trips,fell,causing.*cleaner to_ causing.*ip to_ causing.*...,_ and twist.*_ on.*floor_ on.*ground,employee fell back.*_ ip fell back.*_ worked f...,straps_ drop.*_ fell (.*) bike_ no slip_ no fa...
...,...,...,...,...,...,...
349,vandalism,sever*,NaN,NaN,intentionally.*sever.*others,NaN
350,line of fire,pressure,NaN,NaN,tight.*under.*pressure_tight.*while.*pressur.*...,NaN
351,line of fire,line,NaN,NaN,line of fire,NaN
352,line of fire,release,NaN,NaN,accidental.*release.*pressure,NaN


In [392]:
filter_rules = rules.loc[(rules['group'] == focus_group)]
filter_rules

,group,keyword,rules_pre,rules_post,rules_all,voids
21,competency,train*,NaN,NaN,lack (.*) training_ lack (.*) competency,NaN


In [61]:
keyword = 'bag'
window = 5
doc = 'this is a big bag of shoes. big bags and a second huge back pack bag of rocks'
pre_context = "(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,<window>}<keyword>".replace('<keyword>', str(keyword)).replace('<window>', str(window))      
pre_match = re.findall(pre_context, doc)
pre_match

['this is a big bag', 'of shoes. big bag', 'a second huge back pack bag']

In [62]:
sen_toks = sent_tokenize(doc)

In [64]:
tok_list = sent_tokenize(doc)
tok_list

['this is a big bag of shoes.',
 'big bags and a second huge back pack bag of rocks']

In [69]:
tmp = [re.findall(pre_context, t) for t in tok_list]
tmp

[['this is a big bag'], ['big bag', 'a second huge back pack bag']]

In [71]:
def flatten(list_of_lists):
    return [x for xs in list_of_lists for x in xs]

In [72]:
flatten(tmp)

['this is a big bag', 'big bag', 'a second huge back pack bag']

In [74]:
flatten([re.findall(pre_context, t) for t in tok_list])

['this is a big bag', 'big bag', 'a second huge back pack bag']

In [126]:
syns_data = pd.read_csv('synonyms.csv')
syns_data['syn'] = syns_data['syn'].apply(rbfuncts.replace_syns)
syns_db = syns_data
syns_db

,syn,keywords
0,arm,"arm, bicep, elbow, forearm, finger, fingertip,..."
1,backpain,"bruis, contusion, discomfort, fracturing, graz..."
2,back,back
3,collision,"accident, clipped, collision, collission"
4,competency,"competency, certification, certificate, qualif..."
5,covid,"covid, covid-19, coronavirus, corona virus, pcr"
6,eye,"eye, eyelid, brow, iris, lash, pupil, retina"
7,equipment,"board, box, bucket, drill, hammer, material, n..."
8,failure,"burst, broke, blown off, contain, exiting the,..."
9,fall,"fall, fell, twisted"


In [104]:
def translate_to_regex(rule_part, syns_db):
    """
    converts rule_book rules to regex format;
    drops trailing separators in process
    performs preprocessing to mutate synonyms & expand rule set for all permutations
    """
    if isinstance(rule_part, str): # If rule part is a string...
        
        # 1st, we need to preprocess and mutate synonyms if they are found in rules
        # 1st step is to split the rule set into individual rules
        split_rule = rule_part.split('_ ')
        # create an empty list to hold the mutates rule set
        out_rule = []

        for rule in split_rule:
            
            # Check for presence of synonym, e.g., {eye}
            syns = re.findall("{[a-zA-Z'-]*}", rule)
            # How many do we find?
            syn_count = len(syns)
            # No process based on how many we find (max number is 3, min is 0)
            if syn_count == 1:
                syn_rules = mutate_syn(syns, syns_db, rule, 1)
                out_rule.append(syn_rules)
            elif syn_count == 2:
                syn_rules = mutate_syn(syns, syns_db, rule, 2)
                out_rule.append(syn_rules)
            elif syn_count == 3:
                syn_rules = mutate_syn(syns, syns_db, rule, 3)
                out_rule.append(syn_rules)
            else:       
                # Do nothin, no mutation required as no synonym has been specified
                out_rule.append(rule)
                
        # Join back to single rule set        
        out_rule = '_ '.join(out_rule)
        
        # Now translate to regex
        rule_part = re.sub(r"\s{2,}", " ", out_rule)
        rule_part = re.sub(r'\\\\b', r'\\b', rule_part)
        print(r'(\b' + r'\b)|(\b'.join([s for s in rule_part.split('_ ') if s]) + r'\b)')
        return r'(\b' + r'\b)|(\b'.join([s for s in rule_part.split('_ ') if s]) + r'\b)'
    else:
        return '' 
       

In [105]:
def mutate_syn(syns, syns_db, rule, num):
    syn_rules = []
    if num == 1:    
        syn_raw = syns[0]
        syn = re.sub(r"[\([{})\]]", '', syn_raw)
        found_syn = syns_db.loc[(syns_db['syn'] == syn)]
        for keyword in found_syn.iloc[0, 1].split(', '):
            syn_rules.append(re.sub(syn_raw, keyword, rule))
        syn_rules = '_ '.join(syn_rules)
        return(syn_rules)
    elif num == 2:
        syn_raw_1 = syns[0]
        syn_1 = re.sub(r"[\([{})\]]", '', syn_raw_1)
        found_syn_1 = syns_db.loc[(syns_db['syn'] == syn_1)]

        syn_raw_2 = syns[1]
        syn_2 = re.sub(r"[\([{})\]]", '', syn_raw_2)
        found_syn_2 = syns_db.loc[(syns_db['syn'] == syn_2)]

        for keyword_1 in found_syn_1.iloc[0, 1].split(', '):
            for keyword_2 in found_syn_2.iloc[0, 1].split(', '):
                pass_1 = re.sub(syn_raw_1, keyword_1, rule)
                pass_2 = re.sub(syn_raw_2, keyword_2, pass_1)
                syn_rules.append(pass_2)
        syn_rules = '_ '.join(syn_rules)
        return(syn_rules)
    elif num == 3:
        syn_raw_1 = syns[0]
        syn_1 = re.sub(r"[\([{})\]]", '', syn_raw_1)
        found_syn_1 = syns_db.loc[(syns_db['syn'] == syn_1)]

        syn_raw_2 = syns[1]
        syn_2 = re.sub(r"[\([{})\]]", '', syn_raw_2)
        found_syn_2 = syns_db.loc[(syns_db['syn'] == syn_2)]

        syn_raw_3 = syns[2]
        syn_3 = re.sub(r"[\([{})\]]", '', syn_raw_3)
        found_syn_3 = syns_db.loc[(syns_db['syn'] == syn_3)]

        for keyword_1 in found_syn_1.iloc[0, 1].split(', '):
            for keyword_2 in found_syn_2.iloc[0, 1].split(', '):
                for keyword_3 in found_syn_3.iloc[0, 1].split(', '):
                    pass_1 = re.sub(syn_raw_1, keyword_1, rule)
                    pass_2 = re.sub(syn_raw_2, keyword_2, pass_1)
                    pass_3 = re.sub(syn_raw_3, keyword_3, pass_2)
                    syn_rules.append(pass_3)
        syn_rules = '_ '.join(syn_rules)
        return(syn_rules)   

In [127]:
rules["rules_all"] = [translate_to_regex(x, syns_db) for x in rules["rules_all"]]

(\bslip.*fall\b)|(\bslip.*fell\b)|(\bslip.*twisted\b)|(\bslipped.*fall\b)|(\bslipped.*fell\b)|(\bslipped.*twisted\b)|(\btrip.*fall\b)|(\btrip.*fell\b)|(\btrip.*twisted\b)|(\btripped.*fall\b)|(\btripped.*fell\b)|(\btripped.*twisted\b)|(\bfall\b)|(\bfell\b)|(\btrip\b)|(\btripped\b)|(\bindividual slipped\b)|(\bip slipped\b)|(\bslipped on\b)|(\bhurting foot\b)
(\bfall\b)|(\bfell\b)|(\btrip\b)|(\btripped\b)|(\bindividual slipped\b)|(\bip slipped\b)|(\btripped on\b)|(\bhurting foot\b)
(\bhurting foot\b)
(\bemployee.* uneven.* surface\b)|(\bip.* uneven.* surface\b)|(\bworker.* uneven.* surface\b)|(\bhurting foot\b)
(\bemployee fell back.*\b)|(\bip fell back.*\b)|(\bworked fell back.*\b)|(\bemployee fell over\b)|(\bip fell over\b)|(\bworked fell over*\b)|(\bcontractor.*fell.*ground\b)|(\bemployee.*fell.*ground\b)|(\bhe.*fell.*ground\b)|(\bhis.*fell.*ground\b)|(\bher.*fell.*ground\b)|(\bip.*fell.*ground\b)|(\bman.*fell.*ground\b)|(\bshe.*fell.*ground\b)|(\bleft.*fell.*ground\b)|(\bright.*fell.*

In [210]:
rules["voids"] = [translate_to_regex(x, syns_db) for x in rules["voids"]]

(\bfell (.*) bike\b)|(\bno slip\b)|(\bno fall\b)|(\bit slipped\b)|(\bit fell board slipped\b)|(\btool_slipped plank slipped\b)|(\bpipe slipped\b)|(\bchisel slipped\b)|(\bknife slipped\b)|(\bhand slipped\b)|(\bfalling object\b)|(\bdropped object\b)|(\bhammer.*slip.*\b)|(\b.*hammer.*slip.*\b)|(\brung.*step.*\b)|(\bits\b)|(\bwhile sitting\b)|(\bbus\b)|(\bcar\b)|(\bdigger\b)|(\bdozer\b)|(\bdumper\b)|(\bexcavator\b)|(\bflatbed\b)|(\bforklift\b)|(\bjcb\b)|(\bmanitou\b)|(\bminidigger\b)|(\btruck\b)|(\btrailer\b)|(\btugger\b)|(\butility\b)|(\bvehicle\b)|(\bvan\b)
(\bfell (.*) bike\b)|(\bno slip\b)|(\bno fall\b)|(\bit slipped\b)|(\bit fell board slipped\b)|(\btool_slipped plank slipped\b)|(\bpipe slipped\b)|(\bchisel slipped\b)|(\bknife slipped\b)|(\bhand slipped\b)|(\bgenerator\b)|(\bpower\b)|(\belectricity\b)|(\bfalling object\b)|(\bdropped object\b)|(\brung.*step.*\b)|(\bwhile sitting\b)
(\brung.*step.*\b)|(\bwhile sitting\b)
(\brung.*step.*\b)|(\bwhile sitting\b)
(\bstraps\b)|(\bdrop.*\b)|(

In [211]:
rules["voids"][139]

''

In [100]:
pattern = '(\\bman .* dirt .* eye\\b)'
rbfuncts.check_presence(pattern, 'the man got some dirt in his eye')

True

In [108]:
tmp1 = 'slips'
tmp2 = 'bad'
tmp1 == tmp2

False

In [109]:
doc = 'struck by falling object or equipment, n.e.c. on october 26, 2016, at 12:00 p.m., an employee was using a pry-bar to raise the manholes to the level of the new pavement. the manhole cover slipped off of the pry-bar as he was reaching under the manhole cover and crushed the tip of his left middle finger. the tip of his left middle finger with some bone was amputated at the hospital.'
sen_toks = sent_tokenize(doc)
sen_toks

['struck by falling object or equipment, n.e.c.',
 'on october 26, 2016, at 12:00 p.m., an employee was using a pry-bar to raise the manholes to the level of the new pavement.',
 'the manhole cover slipped off of the pry-bar as he was reaching under the manhole cover and crushed the tip of his left middle finger.',
 'the tip of his left middle finger with some bone was amputated at the hospital.']

In [139]:

check_all = rules["rules_all"][138]
len(check_all)

285673

In [145]:
tmp = pd.DataFrame(check_all.split('|'), columns=['pattern'])
tmp['doc'] = 'on october 26, 2016, at 12:00 p.m., an employee was using a pry-bar to raise the manholes to the level of the new pavement.'
tmp

,pattern,doc
0,(\bcontractor .* abrasion.* .* ankle\b),"on october 26, 2016, at 12:00 p.m., an employe..."
1,(\bcontractor .* abrasion.* .* calf\b),"on october 26, 2016, at 12:00 p.m., an employe..."
2,(\bcontractor .* abrasion.* .* femur\b),"on october 26, 2016, at 12:00 p.m., an employe..."
3,(\bcontractor .* abrasion.* .* foot\b),"on october 26, 2016, at 12:00 p.m., an employe..."
4,(\bcontractor .* abrasion.* .* hamstring\b),"on october 26, 2016, at 12:00 p.m., an employe..."
...,...,...
8735,(\bwoman .* tingling .* knee\b),"on october 26, 2016, at 12:00 p.m., an employe..."
8736,(\bwoman .* tingling .* leg\b),"on october 26, 2016, at 12:00 p.m., an employe..."
8737,(\bwoman .* tingling .* shin\b),"on october 26, 2016, at 12:00 p.m., an employe..."
8738,(\bwoman .* tingling .* toe\b),"on october 26, 2016, at 12:00 p.m., an employe..."


In [174]:
tmp = pd.DataFrame()
for all_ in sen_toks:
    tmp_add = pd.DataFrame(check_all.split('|'), columns=['pattern'])
    tmp_add['doc'] = all_
    tmp = pd.concat([tmp, tmp_add])
tmp

,pattern,doc
0,(\bcontractor .* abrasion.* .* ankle\b),"struck by falling object or equipment, n.e.c."
1,(\bcontractor .* abrasion.* .* calf\b),"struck by falling object or equipment, n.e.c."
2,(\bcontractor .* abrasion.* .* femur\b),"struck by falling object or equipment, n.e.c."
3,(\bcontractor .* abrasion.* .* foot\b),"struck by falling object or equipment, n.e.c."
4,(\bcontractor .* abrasion.* .* hamstring\b),"struck by falling object or equipment, n.e.c."
...,...,...
8735,(\bwoman .* tingling .* knee\b),the tip of his left middle finger with some bo...
8736,(\bwoman .* tingling .* leg\b),the tip of his left middle finger with some bo...
8737,(\bwoman .* tingling .* shin\b),the tip of his left middle finger with some bo...
8738,(\bwoman .* tingling .* toe\b),the tip of his left middle finger with some bo...


In [209]:
import time
find = False
tic = int(round(time.time() * 1000))
for all_ in sen_toks:
    df_scan = pd.DataFrame(check_all.split('|'), columns=['pattern'])
    df_scan['doc'] = all_
    df_scan['finds'] = df_scan[['pattern', 'doc']].apply(lambda x: check_presence(*x), axis=1)
    #df_scan['finds'] = df_scan.apply(lambda x: check_presence(x.pattern, x.doc), axis=1)
    find = bool(sum(df_scan['finds'])) or find  
    if find: break
print(f'Time taken: {int(round(time.time() * 1000)) - tic} ms')    
find

Time taken: 1546 ms


False

In [226]:
check_void = rules["voids"][1]
check_void

'(\\bfell (.*) bike\\b)|(\\bno slip\\b)|(\\bno fall\\b)|(\\bit slipped\\b)|(\\bit fell board slipped\\b)|(\\btool_slipped plank slipped\\b)|(\\bpipe slipped\\b)|(\\bchisel slipped\\b)|(\\bknife slipped\\b)|(\\bhand slipped\\b)|(\\bgenerator\\b)|(\\bpower\\b)|(\\belectricity\\b)|(\\bfalling object\\b)|(\\bdropped object\\b)|(\\brung.*step.*\\b)|(\\bwhile sitting\\b)'

In [227]:
check_void.split('|')

['(\\bfell (.*) bike\\b)',
 '(\\bno slip\\b)',
 '(\\bno fall\\b)',
 '(\\bit slipped\\b)',
 '(\\bit fell board slipped\\b)',
 '(\\btool_slipped plank slipped\\b)',
 '(\\bpipe slipped\\b)',
 '(\\bchisel slipped\\b)',
 '(\\bknife slipped\\b)',
 '(\\bhand slipped\\b)',
 '(\\bgenerator\\b)',
 '(\\bpower\\b)',
 '(\\belectricity\\b)',
 '(\\bfalling object\\b)',
 '(\\bdropped object\\b)',
 '(\\brung.*step.*\\b)',
 '(\\bwhile sitting\\b)']

In [245]:
doc = 'my contractor suffered an abrasion to his ankle. and was struck by falling object or equipment, n.e.c. on october 26, 2016, at 12:00 p.m., an employee was using a pry-bar to raise the manholes to the level of the new pavement. the manhole cover slipped off of the pry-bar as he was reaching under the manhole cover and crushed the tip of his left middle finger. the tip of his left middle finger with some bone was amputated at the hospital.'
sen_toks = sent_tokenize(doc)
sen_toks

['my contractor suffered an abrasion to his ankle.',
 'and was struck by falling object or equipment, n.e.c.',
 'on october 26, 2016, at 12:00 p.m., an employee was using a pry-bar to raise the manholes to the level of the new pavement.',
 'the manhole cover slipped off of the pry-bar as he was reaching under the manhole cover and crushed the tip of his left middle finger.',
 'the tip of his left middle finger with some bone was amputated at the hospital.']

In [246]:
void = False
for all_ in sen_toks:
    df_scan = pd.DataFrame(check_all.split('|'), columns=['pattern'])    
    df_scan['doc'] = all_
    df_scan['finds'] = df_scan[['pattern', 'doc']].apply(lambda x: check_presence(*x), axis=1)    
    #df_scan['finds'] = df_scan.apply(lambda x: check_presence(x.pattern, x.doc), axis=1)
    find = bool(sum(df_scan['finds']))
    if len(check_void) != 0:
        df_void = pd.DataFrame(check_void.split('|'), columns=['pattern'])
        df_void['doc'] = all_
        df_void['voids'] = df_void[['pattern', 'doc']].apply(lambda x: check_presence(*x), axis=1)
        void = bool(sum(df_void['voids']))
    final_match = find and not void    
    if final_match: break
final_match

True

In [199]:
any([check_presence(check_pre, pre) for pre in pre_match])

True

In [269]:
def check_apply(sens, rules):
    if len(rules) != 0:
        for all_ in sens:
            print(all_)
            df_scan = pd.DataFrame(rules.split('|'), columns=['pattern'])    
            df_scan['doc'] = all_
            df_scan['finds'] = df_scan[['pattern', 'doc']].apply(lambda x: check_presence(*x), axis=1)    
            find = bool(sum(df_scan['finds']))       
            if find: break
    else:
        find = False
    print(find)    
    return(find)

In [284]:
def check_apply(sen, rules):
    if len(rules) != 0:
        df_scan = pd.DataFrame(rules.split('|'), columns=['pattern'])    
        df_scan['doc'] = sen
        df_scan['finds'] = df_scan[['pattern', 'doc']].apply(lambda x: check_presence(*x), axis=1)    
        find = bool(sum(df_scan['finds']))       
    else:
        find = False   
    return(find)

In [285]:
def check_apply_all(sens, check_pre, check_post, check_all, check_void):
    for sen in sens:
        pre_check, post_check, all_check, void_check = False, False, False, False
        if len(check_pre) != 0:
            pre_check = check_apply(sen, check_pre)
        if len(check_post) != 0:
            post_check = check_apply(sen, check_post)
        if len(check_all) != 0:
            all_check = check_apply(sen, check_all)
        if len(check_void) != 0:
            void_check = check_apply(sen, check_void)    

        final_match = (pre_check or post_check or all_check) and not void_check
        print(final_match)
        if final_match: break
    return(final_match)

In [286]:
check_apply_all(sen_toks, check_pre=check_all, check_post=check_all, check_all=check_all, check_void=check_void)

True


True

In [272]:
True and not True

False

In [257]:
pre_check

False

In [253]:
check_apply(sen_toks, check_all)

True

In [251]:
check_apply(sen_toks, check_void)

True

In [178]:
tmp['finds'] = tmp.apply(lambda x: check_presence(x.pattern, x.doc), axis=1)

In [176]:
sum(tmp['finds'])

0

In [179]:
tmp['finds'] = tmp[['pattern', 'doc']].apply(lambda x: check_presence(*x), axis=1)
tmp

,pattern,doc,finds
0,(\bcontractor .* abrasion.* .* ankle\b),"struck by falling object or equipment, n.e.c.",False
1,(\bcontractor .* abrasion.* .* calf\b),"struck by falling object or equipment, n.e.c.",False
2,(\bcontractor .* abrasion.* .* femur\b),"struck by falling object or equipment, n.e.c.",False
3,(\bcontractor .* abrasion.* .* foot\b),"struck by falling object or equipment, n.e.c.",False
4,(\bcontractor .* abrasion.* .* hamstring\b),"struck by falling object or equipment, n.e.c.",False
...,...,...,...
8735,(\bwoman .* tingling .* knee\b),the tip of his left middle finger with some bo...,False
8736,(\bwoman .* tingling .* leg\b),the tip of his left middle finger with some bo...,False
8737,(\bwoman .* tingling .* shin\b),the tip of his left middle finger with some bo...,False
8738,(\bwoman .* tingling .* toe\b),the tip of his left middle finger with some bo...,False


In [157]:
finds = []
for i in range(len(tmp)):
    finds.append(check_presence(tmp['pattern'][i],tmp['doc'][i]))
sum(finds)

0

In [180]:
all_check = any([check_presence(check_all, all_) for all_ in sen_toks])
void_check = False
#void_check = any([check_presence(check_void, all_) for all_ in sen_toks])
final_match = all_check and not void_check
final_match

False

In [138]:
for all_ in sen_toks:
    print(all_)
    print(check_presence(check_all, all_))

struck by falling object or equipment, n.e.c.
False
on october 26, 2016, at 12:00 p.m., an employee was using a pry-bar to raise the manholes to the level of the new pavement.
False
the manhole cover slipped off of the pry-bar as he was reaching under the manhole cover and crushed the tip of his left middle finger.
False
the tip of his left middle finger with some bone was amputated at the hospital.
False


In [365]:
rules = pd.read_csv('data/rule_book_kwic.csv')
rules["keyword"] = [x.replace("*", "[a-zA-Z'-]*") + r"\b" for x in rules["keyword"]]
rules["rules_pre"] = [translate_to_regex(x, syns_db) for x in rules["rules_pre"]]
rules["rules_post"] = [translate_to_regex(x, syns_db) for x in rules["rules_post"]]
rules["rules_all"] = [translate_to_regex(x, syns_db) for x in rules["rules_all"]]
rules["voids"] = [translate_to_regex(x, syns_db) for x in rules["voids"]]

(\bwalking (.*) snow\b)|(\bwalking (.*) ice\b)|(\bwalking (.*) wet\b)|(\bcausing.*cleaner to\b)|(\bwalking when\b)|(\bclimbing.* when\b)|(\blady\b)|(\bman\b)|(\bwroker\b)|(\bemployee\b)|(\bip\b)|(\bfoot\b)|(\bleg\b)
(\bwalking (.*) snow\b)|(\bwalking (.*) ice\b)|(\bwalking (.*) wet\b)|(\bcausing.*cleaner to\b)|(\bwalking when\b)|(\bclimbing.* when\b)|(\blady\b)|(\bman\b)|(\bwroker\b)|(\bemployee\b)|(\bip\b)
(\bcausing.*cleaner to\b)|(\bcausing.*ip to\b)|(\bcausing.*employee to_causing.*worker to\b)|(\bwalking.*when\b)|(\bclimbing.* when\b)|(\blady\b)|(\bman\b)|(\bwroker\b)|(\bemployee\b)|(\bip\b)
(\bcausing.*cleaner to\b)|(\bcausing.*ip to\b)|(\bcausing.*employee to_causing.*worker to\b)|(\bwalking.*when\b)|(\bclimbing.* when\b)
(\bcausing.*cleaner to\b)|(\bcausing.*ip to\b)|(\bcausing.*employee to_causing.*worker to\b)|(\bwalking.*when\b)|(\bclimbing.* when\b)|(\blady\b)|(\bman\b)|(\bwroker\b)|(\bemployee\b)|(\bip\b)
(\bloss of\b)|(\blost his\b)|(\blost her\b)|(\blost their\b)
(\bsome

In [366]:
rules.loc[(rules.group == 'general illness or health issue')]

,group,keyword,rules_pre,rules_post,rules_all,voids
196,general illness or health issue,ach[a-zA-Z'-]*\b,,,(\bstomach (.*) aching\b)|(\bstomach (.*) ach\...,
197,general illness or health issue,heart\b,(\bpassed out\b)|(\bfelt\b)|(\bstruggling with\b),(\b passed out\b),(\bheart attack\b),
198,general illness or health issue,unwell\b,(\bpassed out\b)|(\bfelt\b)|(\bcomplained\b)|(...,(\b passed out\b),(\bunwell and fainted\b)|(\bunwell (.*) fainte...,
199,general illness or health issue,well\b,(\bdid not feel\b)|(\bdidn't feel\b)|(\bdidnt ...,,,
200,general illness or health issue,flu\b,(\bdiagnosed.*with\b)|(\bstruggling with\b),,(\bflu (.*) symptoms\b)|(\bunwell and fainted\...,
201,general illness or health issue,motion\b,(\bfeeling unwell\b)|(\bfelt unwell\b),(\bsickness.*\b),,
202,general illness or health issue,chest\b,(\bsore\b)|(\bsuffer.*\b)|(\bstruggling with\b),,(\bcomplained.*pain\b)|(\bexperienced.*pain\b)...,
203,general illness or health issue,breath[a-zA-Z'-]*\b,(\bstruggling with\b),,(\bbreath.* difficulties\b)|(\bbreath.* issue*...,
204,general illness or health issue,cramp[a-zA-Z'-]*\b,(\bsore\b)|(\bsuffer.*\b)|(\bstruggling with\b),,(\bcramp.* in.* leg.*\b)|(\bcramp.* in.* arm.*...,
205,general illness or health issue,throat\b,(\bsore\b)|(\bsuffer.*\b)|(\bstruggling with\b),(\bsore\b),,


In [394]:
i = 224
keyword=rules["keyword"][i]
check_pre=rules["rules_pre"][i]
check_post=rules["rules_post"][i]
check_all=rules["rules_all"][i]
check_void=rules["voids"][i]

In [395]:
#keyword = "collap[a-zA-Z'-]*\b"
#keyword = "collapsed"
window = 12
pre_context = "(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,<window>}<keyword>".replace('<keyword>', str(keyword)).replace('<window>', str(window))
post_context = "<keyword>(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,<window>}".replace('<keyword>', str(keyword)).replace('<window>', str(window))
all_context = "(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,<window>}<keyword>(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,<window>}".replace('<keyword>', str(keyword)).replace('<window>', str(window))

In [396]:
pre_context

"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,12}collap[a-zA-Z'-]*\\b"

In [397]:
post_context

"collap[a-zA-Z'-]*\\b(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,12}"

In [398]:
all_context

"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,12}collap[a-zA-Z'-]*\\b(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,12}"

In [399]:
doc = 'ip collapsed while walking on the airplane from the bathroom. my contractor suffered an abrasion to his ankle. and was struck by falling object or equipment, n.e.c. on october 26, 2016, at 12:00 p.m., an employee was using a pry-bar to raise the manholes to the level of the new pavement. the manhole cover slipped off of the pry-bar as he was reaching under the manhole cover and crushed the tip of his left middle finger. the tip of his left middle finger with some bone was amputated at the hospital.'
sen_toks = sent_tokenize(doc)
sen_toks
pre_match = flatten([re.findall(pre_context, t) for t in sen_toks]) 
post_match = flatten([re.findall(post_context, t) for t in sen_toks])
all_match = flatten([re.findall(all_context, t) for t in sen_toks])

In [400]:
pre_match

['ip collapsed']

In [401]:
post_match

['collapsed while walking on the airplane from the bathroom']

In [402]:
all_match

['ip collapsed while walking on the airplane from the bathroom']

In [403]:
check_all

'(\\bcontractor collapsed .* walking\\b)|(\\bemployee collapsed .* walking\\b)|(\\bhe collapsed .* walking\\b)|(\\bhis collapsed .* walking\\b)|(\\bher collapsed .* walking\\b)|(\\bip collapsed .* walking\\b)|(\\bman collapsed .* walking\\b)|(\\bshe collapsed .* walking\\b)|(\\bleft collapsed .* walking\\b)|(\\bright collapsed .* walking\\b)|(\\boperator collapsed .* walking\\b)|(\\boperative collapsed .* walking\\b)|(\\bperson collapsed .* walking\\b)|(\\bscaffolder collapsed .* walking\\b)|(\\bsubcontractor collapsed .* walking\\b)|(\\btechnician collapsed .* walking\\b)|(\\bteam member collapsed .* walking\\b)|(\\bworker collapsed .* walking\\b)|(\\bwoman collapsed .* walking\\b)|(\\bcontractor collapsed .* working\\b)|(\\bemployee collapsed .* working\\b)|(\\bhe collapsed .* working\\b)|(\\bhis collapsed .* working\\b)|(\\bher collapsed .* working\\b)|(\\bip collapsed .* working\\b)|(\\bman collapsed .* working\\b)|(\\bshe collapsed .* working\\b)|(\\bleft collapsed .* working\\b)|

In [404]:
check_pre

''

In [405]:
def check_apply_all(sens_pre, sens_post, sens_all, check_pre, check_post, check_all, check_void):
    pre_check, post_check, all_check, void_check = False, False, False, False
    for sen in sens_pre:
        pre_check = False
        if len(check_pre) != 0:
            pre_check = check_apply(sen, check_pre)
            if pre_check: break
            
    for sen in sens_post:
        post_check = False        
        if len(check_post) != 0:
            post_check = check_apply(sen, check_post)
            if post_check: break
            
    for sen in sens_all:    
        all_check, void_check = False, False   
        if len(check_all) != 0:
            all_check = check_apply(sen, check_all)
        if len(check_void) != 0:
            void_check = check_apply(sen, check_void)   
            if void_check: break 

    final_match = (pre_check or post_check or all_check) and not void_check
        
    return(final_match)

In [406]:
check_apply_all(
            sens_pre=pre_match, 
            sens_post=post_match, 
            sens_all=all_match, 
            check_pre=check_pre, 
            check_post=check_post, 
            check_all=check_all, 
            check_void=check_void
            )

True